## Constant from IV-characteristics : <font color = 'sky blue'> $n_e, T_e, T_-, I_{sat+}$ </font>
<font size="2">
    We use classical Langmuir probe method here to obtain constants listed above. <br><br>
    $n_e=\frac{I(V_p)}{eA} (\frac{2\pi m_e}{T_e})^{1/2} $ <br><br>
    $I(V_p)$ is probe current at plasma potential, $A$ is probe surface area.<br><br>
    $T_e = \frac{4\sqrt{2m_e/e}}{3n_eA} \int_{0}^{-\infty} I''(V) {|V|}^{3/2} dV$ <br><br>
    We can get $I''(V)$ from second derivative and $I_{sat+}$ from most negative probe potential. $T_-$ would be given value. <br><br>
    
</font>

In [14]:
ne  = 1
Te = 2
Tm = 3
Isatp = 4

## Iteration method : <font color = 'purple'> </font>
<font size = '2'>
    Initial value : <font color = 'sky blue'> $\alpha_{0,0} = 2, r_{sh,0} = 0$ </font><br><br>
    Constant value : <font color = 'sky blue'> $n_e, T_e, T_-, I_{sat+}, h_r$ </font><br><br>
    We can also define $\alpha_{s,0}$ by relation between $\alpha_0$ and $\alpha_s$<br><br>
    Update modified Bohm speed : $u_{B+,k}^* = \sqrt{\frac{e T_e}{m_+}}\sqrt{\frac{1+\alpha_{s,k-1}}{1+\gamma \alpha_{s,k-1}}}$<br><br>
    Update ion density : $n_{+,k} = \frac{I_{sat+}}{h_r S_{eff,k-1} e u_{B+,k}^*}$ (Bohm flux) <br><br>
    Update electronegativity : $\alpha_{0,k} = \frac{n_{+,k}}{n_e}-1$ <br><br>
    Update $\alpha_{s,k}$ : $\alpha_{0,k} = \alpha_{s,k} exp(-\frac{1}{2} (\frac{1+\alpha_{s,k}}{1+\gamma \alpha_{s,k}})(1-\gamma))$<br><br>
    Update positive ion current density : $J_{+,k} = h_r e n_{+,k}u_{B+,k}^*$ <br><br>
    Update effective ion collecting area : $S_{eff,k} = 2\pi s_k l$ where $s_k=r_{sh,k}+a$ 이파트 이상함<br><br> 
    From this point, all values are re-injected to Modified Bohm speed equation for new iteration. <br><br>
    Iteration is stopped when $|\alpha_{0,k-1}-\alpha_{0,k}|<0.01\alpha_{0,k}$ <br><br>
    $S_{eff}(V) = 2\pi l r_{sh}(V) + 2\pi r_{sh}^2(V)$ <br><br>
</font>

In [25]:
class iter_method:
    def __init__(self,ne,Te,Tm,Isatp):
        self.ne = ne
        self.Te = Te
        self.Tm = Tm
        self.Isatp = Isatp

In [24]:
sample = iter_method(ne, Te, Tm, Isatp)